In [1]:
cd py-hanspell-master


c:\Users\SSAFY\Desktop\hanspell\py-hanspell-master


In [2]:
!python setup.py install


running install
running bdist_egg
running egg_info
writing py_hanspell.egg-info\PKG-INFO
writing dependency_links to py_hanspell.egg-info\dependency_links.txt
writing requirements to py_hanspell.egg-info\requires.txt
writing top-level names to py_hanspell.egg-info\top_level.txt
reading manifest file 'py_hanspell.egg-info\SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'py_hanspell.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
creating build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\hanspell
copying build\lib\hanspell\constants.py -> build\bdist.win-amd64\egg\hanspell
copying build\lib\hanspell\response.py -> build\bdist.win-amd64\egg\hanspell
copying build\lib\hanspell\spell_checker.py -> build\bdist.win-amd64\egg\hanspell
copying build\lib\hanspell\__init__.py -> build\bdist.win-amd64\egg\hanspell
byte-compiling build\bdist.win-amd64\egg\hanspell\constants.py to constants.cpython-311.pyc
by

c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  warnings.warn(
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
c:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\command\easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
zip_safe flag not set; analyzing archive contents...
error: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'c:\\users\\ssafy\\appdata\\local\\programs\\python\\python311\\lib\\site-packages\\py_hanspell-1.1-py3.11.egg'


In [3]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\ssafy\appdata\local\programs\python\python311\lib\site-packages\certifi-2023.7.22-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\ssafy\appdata\local\programs\python\python311\lib\site-packages\charset_normalizer-3.3.1-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\ssafy\appdata\local\programs\python\python311\lib\site-packages\idna-3.4-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg a

In [14]:
from hanspell import spell_checker

# -*- coding: utf-8 -*-
from collections import namedtuple
import requests
import json
import time
import sys
from collections import OrderedDict
import xml.etree.ElementTree as ET

base_url = 'https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy'

class CheckResult:
    PASSED = 0
    WRONG_SPELLING = 1
    WRONG_SPACING = 2
    AMBIGUOUS = 3
    STATISTICAL_CORRECTION = 4

_checked = namedtuple('Checked',
    ['result', 'original', 'checked', 'errors', 'words', 'time'])

class Checked(_checked):
    def __new__(cls, result=False, original='', checked='', errors=0, words=[], time=0.0):
        return super(Checked, cls).__new__(
            cls, result, original, checked, errors, words, time)

    def as_dict(self):
        d = {
            'result': self.result,
            'original': self.original,
            'checked': self.checked,
            'errors': self.errors,
            'words': self.words,
            'time': self.time,
        }
        return d

    def only_checked(self):
        return self.checked

_agent = requests.Session()
PY3 = sys.version_info[0] == 3

def _remove_tags(text):
    text = u'<content>{}</content>'.format(text).replace('<br>','')
    if not PY3:
        text = text.encode('utf-8')

    result = ''.join(ET.fromstring(text).itertext())

    return result


def check(text):
    if isinstance(text, list):
        result = []
        for item in text:
            checked = check(item)
            result.append(checked)
        return result
    # 300 자로 수정함
    if len(text) > 300:
        return Checked(result=False)
    
    # passportkey, callback 직접 가져옴
    payload = { 
        'passportKey': 'f293ec23e93c390c5ca0bbff1ab129da6f96ae75', 
        '_callback': 'jQuery11240276810914717724_1698049775981',
        'q': text,
        'color_blindness': '0' } 


    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'referer': 'https://search.naver.com/',
    }

    start_time = time.time()
    r = _agent.get(base_url, params=payload, headers=headers)
    passed_time = time.time() - start_time
    r = r.text
    json_str = r[r.find('{'): -2]
    json_data = json.loads(json_str)
    data = json_data['message']['result']['html']
    result = {
        'result': True,
        'original': text,
        'checked': _remove_tags(data),
        'errors': json_data['message']['result']['errata_count'],
        'time': passed_time,
        'words': OrderedDict(),
    }

    data = data.replace('<em class=\'green_text\'>', '<green>') \
               .replace('<em class=\'red_text\'>', '<red>') \
               .replace('<em class=\'violet_text\'>', '<violet>') \
               .replace('<em class=\'blue_text\'>', '<blue>') \
               .replace('</em>', '<end>')
    items = data.split(' ')
    words = []
    tmp = ''
    for word in items:
        if tmp == '' and word[:1] == '<':
            pos = word.find('>') + 1
            tmp = word[:pos]
        elif tmp != '':
            word = u'{}{}'.format(tmp, word)
        
        if word[-5:] == '<end>':
            word = word.replace('<end>', '')
            tmp = ''

        words.append(word)

    for word in words:
        check_result = CheckResult.PASSED
        if word[:5] == '<red>':
            check_result = CheckResult.WRONG_SPELLING
            word = word.replace('<red>', '')
        elif word[:7] == '<green>':
            check_result = CheckResult.WRONG_SPACING
            word = word.replace('<green>', '')
        elif word[:8] == '<violet>':
            check_result = CheckResult.AMBIGUOUS
            word = word.replace('<violet>', '')
        elif word[:6] == '<blue>':
            check_result = CheckResult.STATISTICAL_CORRECTION
            word = word.replace('<blue>', '')
        result['words'][word] = check_result

    result = Checked(**result)

    return result

if '__main__':
    text = "15알에성원이랑떡보끼"
    hanspell_sent = check(text)
    print(hanspell_sent.checked)

15알에 성원이랑 떡볶이
